1. Строим словарь из текстов на lenta.ru https://cloud.mail.ru/public/857B/uheFJxvHF
2. Реализуем функцию подсчета расстояния Левенштейна
3. Делаем спелчекер с помощью поиска ближайших по Левенштейну слов
4. Делаем спелчекер путем генерации ближайших слов

In [4]:
from tqdm import tqdm_notebook
from collections import Counter

In [39]:
def distance(a, b):
    "Calculates the Levenshtein distance between a and b."
    n, m = len(a), len(b)
    if n > m:
        a, b = b, a
        n, m = m, n

    current_row = range(n+1)
    for i in range(1, m+1):
        previous_row, current_row = current_row, [i]+[0]*n
        for j in range(1,n+1):
            add, delete, change = previous_row[j]+1, current_row[j-1]+1, previous_row[j-1]
            if a[j-1] != b[i-1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

3. Делаем спелчекер с помощью поиска ближайших по Левенштейну слов

_____________

In [6]:
input_words = ['путн', 'оцинил', 'роботу', 'новвых', 'самалетав', 'и', 'виртолтов', 'в', 'сирийи']

In [7]:
max_len = 0
for in_word in input_words:
    max_len = max(max_len, len(in_word))

print(max_len)

9


In [8]:
words_counter = Counter()

In [9]:
%%time

with open('./lenta_words.txt', 'r') as fin:
    words_counter.update(fin)

CPU times: user 18.1 s, sys: 948 ms, total: 19.1 s
Wall time: 20.1 s


In [16]:
len(words_counter)

463163

In [12]:
%%time

min_threshold = 100
words_set_looking = {x.strip() : words_counter[x] for x in words_counter if words_counter[x] >= 100}

CPU times: user 300 ms, sys: 4.78 ms, total: 305 ms
Wall time: 314 ms


In [14]:
words_set_looking

{'23': 11483,
 '43': 2358,
 '26': 8497,
 'января': 12888,
 '2000': 5156,
 'едет': 105,
 'на': 351071,
 'переговоры': 3608,
 'с': 192960,
 'продолжить': 652,
 'собирается': 1971,
 'глава': 10118,
 'госсовета': 167,
 'республики': 5961,
 'он': 58106,
 'договориться': 641,
 'ними': 1726,
 'о': 84405,
 'оружия': 3705,
 'по': 198525,
 'информации': 8781,
 'интерфакса': 539,
 'отправился': 535,
 'в': 858655,
 'чечню': 245,
 'согласия': 480,
 'министерства': 3236,
 'обороны': 5190,
 'и': 402766,
 'других': 8420,
 'федеральных': 973,
 'силовых': 720,
 'ведомств': 454,
 'рассчитывает': 612,
 'лично': 1440,
 'встретиться': 482,
 'президентом': 2962,
 'обсудить': 805,
 'как': 88256,
 'лучше': 1865,
 'ему': 10197,
 'поступить': 291,
 'для': 62467,
 'того': 31495,
 'чтобы': 18989,
 'война': 1133,
 'чечне': 1379,
 'была': 19815,
 'прекращена': 124,
 'у': 40778,
 'меня': 3949,
 'есть': 13043,
 'конкретный': 112,
 'план': 1864,
 'этот': 11391,
 'счет': 4077,
 'утверждает': 2705,
 'подробности': 1091,


In [23]:
min_dists = [1e4] * len(input_words)
cor_words = [[] for i in range(len(input_words))]

In [24]:
for ex_word, j in tqdm_notebook(words_set_looking.items()):
    for i, in_word in enumerate(input_words, 0):
        d = distance(ex_word, in_word)
        if d < min_dists[i]:
            min_dists[i] = d
            cor_words[i] = [(ex_word, j)]
        elif d == min_dists[i]:
            cor_words[i].append((ex_word, j))

In [25]:
cor_words

[[('путь', 963), ('пути', 1664), ('путин', 5709)],
 [('оценил', 437)],
 [('работу', 5058), ('робота', 220), ('робот', 168), ('роботы', 116)],
 [('новых', 4083)],
 [('самолета', 2995), ('самолетов', 2654), ('самолетах', 244)],
 [('и', 402766)],
 [('вертолетов', 912)],
 [('в', 858655)],
 [('сирии', 2158)]]

In [40]:
for i, in_word in enumerate(input_words, 0):
    print(in_word, '->', max(cor_words[i], key=lambda x: x[1])[0], sep='\t')

путн	->	путин
оцинил	->	оценил
роботу	->	работу
новвых	->	новых
самалетав	->	самолета
и	->	и
виртолтов	->	вертолетов
в	->	в
сирийи	->	сирии
